In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AutoConfig


/home2/advaith.malladi/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)


In [3]:
promptuned_trans_model = "./translation_best"
promptuned_qa_model = "./qa_best"
promptuned_summ_model = "./summarisation_best"



In [5]:
import os
# print cwd
print(os.getcwd())
path = "./models/summ_pt.pt"
summ_test = torch.load(path)

/home2/advaith.malladi


In [6]:
sent = " I hope to god this works"
inputs = tokenizer(sent, return_tensors="pt")
output = summ_test(**inputs)

In [8]:
summ_test.save_pretrained("summ_test")

In [9]:
# open promptuned model directory and load gpt2 model 
from transformers import GPT2Model, GPT2LMHeadModel, GPT2Config
qa_config = AutoConfig.from_pretrained(promptuned_qa_model)
qa_model = GPT2Model.from_pretrained(promptuned_qa_model, config=qa_config)
qa_model.eval()
summ_config = AutoConfig.from_pretrained("summ_test")
summ_model = GPT2Model.from_pretrained(promptuned_summ_model, config=summ_config)
summ_model.eval()
trans_config = AutoConfig.from_pretrained(promptuned_trans_model)
trans_model = GPT2Model.from_pretrained(promptuned_trans_model, config=trans_config)
trans_model.eval()



/home2/advaith.malladi/miniconda3/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [10]:
sent = "I want to test if this model will add the tokens by itself"
input_ids = tokenizer.encode(sent, return_tensors="pt")
outputs_qa = qa_model(input_ids)
outputs_summ = summ_model(input_ids)
outputs_trans = trans_model(input_ids)
print(outputs_qa['last_hidden_state'].shape)
print(outputs_summ['last_hidden_state'].shape)
print(outputs_trans['last_hidden_state'].shape)




torch.Size([1, 13, 768])
torch.Size([1, 13, 768])
torch.Size([1, 13, 768])


In [11]:
from tqdm import tqdm
import numpy as np
def get_representations(list_of_sents):
    summ_reps = []
    qa_reps = []
    trans_reps = []
    for sent in tqdm(list_of_sents, total = len(list_of_sents)):
        #sent = " ".join(sent)
        #print(sent)
        input_ids = tokenizer.encode(sent, return_tensors="pt")
        #print(input_ids[0].shape)
        #print("ch1")
        outputs_qa = qa_model(input_ids)
        #print("ch2")
        outputs_summ = summ_model(input_ids)
        #print("ch3")
        outputs_trans = trans_model(input_ids)
        #print("ch4")
        outputs_qa = outputs_qa['last_hidden_state']
        outputs_summ = outputs_summ['last_hidden_state']
        outputs_trans = outputs_trans['last_hidden_state']
        #print("ch5")
        # get mean_pooled representations of each one
        summ_pooled = torch.mean(outputs_summ, dim=1)
        qa_pooled = torch.mean(outputs_qa, dim=1)
        trans_pooled = torch.mean(outputs_trans, dim=1)
        #print("ch6")
        summ_pooled = np.array(summ_pooled.detach().numpy())
        qa_pooled = np.array(qa_pooled.detach().numpy())
        trans_pooled = np.array(trans_pooled.detach().numpy())
        # reshape from 1,x to x
        summ_pooled = summ_pooled.reshape(-1)
        qa_pooled = qa_pooled.reshape(-1)
        trans_pooled = trans_pooled.reshape(-1)
        summ_reps.append(summ_pooled)
        qa_reps.append(qa_pooled)
        trans_reps.append(trans_pooled)
    return summ_reps, qa_reps, trans_reps


In [32]:
sent1 = "I am here"
sent2 = "hi in test right now"
sent3 = "lets see if it works"
list_sents = [sent1, sent2, sent3]
summ_reps, qa_reps, trans_reps = get_representations(list_sents)

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  5.22it/s]


In [14]:
# print cwd
import os
print(os.getcwd())
working_dir = "csai_a5"
# switch to working dir
os.chdir(working_dir)

/home2/advaith.malladi


In [15]:

parent_dir = "assignment5"
stim_file = "stimuli.txt"
subj1 = "subj1.npy"
subj2 = "subj2.npy"


In [16]:
# load the stims file
import regex as re
with open(os.path.join(parent_dir, stim_file), "r") as f:
    stims = f.readlines()
final_stims = []
for item in stims:
    # remove all numbers and punctuation
    item = re.sub(r'[^\w\s]','',item)
    # remove all numbers
    item = re.sub(r'\d+', '', item)
    item = item.lower().split()
    final_stims.append(item)


In [17]:
summ_reps, qa_reps, trans_reps = get_representations(final_stims)

100%|█████████████████████████████████████████| 627/627 [00:33<00:00, 18.54it/s]


In [18]:
# load fmri data of subj1 and subj2
import numpy as np
subj1_data = np.load(os.path.join(parent_dir, subj1), allow_pickle=True).item()
subj2_data = np.load(os.path.join(parent_dir, subj2), allow_pickle=True).item()


In [19]:

subj1_lang = subj1_data["language"]
subj1_vis = subj1_data["vision"]
subj1_dmn = subj1_data["dmn"]
subj1_task = subj1_data["task"]
subj2_lang = subj2_data["language"]
subj2_vis = subj2_data["vision"]
subj2_dmn = subj2_data["dmn"]
subj2_task = subj2_data["task"]
print(subj1_lang.shape, subj1_vis.shape, subj1_dmn.shape, subj1_task.shape, subj2_lang.shape, subj2_vis.shape, subj2_dmn.shape, subj2_task.shape)

(627, 11437) (627, 33792) (627, 17190) (627, 35120) (627, 10791) (627, 31109) (627, 15070) (627, 30594)


In [20]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
# import R2 as well
from sklearn.metrics import r2_score
def cos_d(y1, y2):
    cos_sim = np.dot(y1, y2)/(np.linalg.norm(y1)*np.linalg.norm(y2))
    cos_d = 1 - cos_sim
    return cos_d
def get_2v2_accuracy(y_pred, y_test):
    N = y_pred.shape[0]
    tot_cnt = 0
    pos_cnt = 0
    for i in range(0, N):
        for j in range(1, N):
            yi_test = y_test[i]
            yi_pred = y_pred[i]
            yj_test = y_test[j]
            yj_pred = y_pred[j]
            score1 = cos_d(yi_test, yi_pred) + cos_d(yj_test, yj_pred)
            score2 = cos_d(yi_test, yj_pred) + cos_d(yj_test, yi_pred)
            if score1 < score2:
                pos_cnt += 1
            tot_cnt += 1
    return pos_cnt/tot_cnt

def get_avg_corr(y_pred, y_test):
    N = y_pred.shape[0]
    tot_corr = 0
    for i in range(0, N):
        corr = np.corrcoef(y_pred[i], y_test[i])[0,1]
        tot_corr += corr
    return tot_corr/N

def decoding_module(X, y):
    kf = KFold(n_splits=5)
    mse = []
    r2 = []
    acc = []
    acc_scores = []
    corr_scores = []
    for train_index, test_index in  kf.split(X) :
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = Ridge()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        get_2v2_accuracy(y_pred, y_test)
        mse.append(mean_squared_error(y_test, y_pred))
        r2.append(r2_score(y_test, y_pred))
        acc_scores.append(get_2v2_accuracy(y_pred, y_test))
        corr_scores.append(get_avg_corr(y_pred, y_test))
    loc_dict = {}
    loc_dict["MSE"] = np.mean(mse)
    loc_dict["R2"] = np.mean(r2)
    loc_dict["Accuracy"] = np.mean(acc_scores)
    loc_dict["Correlation"] = np.mean(corr_scores)
    return loc_dict

    


subj1_decoding_scores = {}
subj2_decoding_scores = {}

In [21]:
x = np.array(summ_reps)
y = subj1_lang
decoding_scores = decoding_module(x, y)

In [23]:
print(decoding_scores)

{'MSE': 25.365624188836136, 'R2': -0.4493539880303308, 'Accuracy': 0.6715242191500256, 'Correlation': 0.6844155329498589}


In [24]:
x = np.array(trans_reps)
y = subj1_lang
decoding_scores = decoding_module(x, y)
print(decoding_scores)

{'MSE': 25.365624188836136, 'R2': -0.4493539880303308, 'Accuracy': 0.6715242191500256, 'Correlation': 0.6844155329498589}


In [25]:
print((summ_reps[0].shape))

(768,)


In [79]:
path = "./models/summ_pt.pt"
summ_test = torch.load(path)
qna_path = "./models/qna-fine-tuned.pt"
qna_test = torch.load(qna_path)

In [81]:
sent = " I hope this works"
sent2 = " I just want to go home as soon as possible"
list_s = [sent, sent2]
max_length = 20
tokenizer.pad_token = tokenizer.eos_token
fin_in = []
for sent in list_s:
    input_s = tokenizer.encode(sent,return_tensors="pt", max_length=max_length, truncation=True, padding="max_length")
    prompt = [0, 1, 2]
    # convert to tensor
    prompt = torch.tensor(prompt)
    # add batch dimension 1
    prompt = prompt.unsqueeze(0)
    input_s = torch.cat((prompt, input_s), 1)
    fin_in.append(input_s)
    


In [90]:
fin_in1 = fin_in
output = summ_test(fin_in[1])
# print device of model and fin_in
qna_test = qna_test.to("cpu")
output1 = qna_test(fin_in[1])

In [91]:
print(len(output[1]))
summ_row = 0
for row in output[1]:
    for loc_row in row:
        # shape of loc row is 1, a, b, c
        # reshape it to 1, b, a*c
        reshaped = loc_row.reshape(1, loc_row.shape[2], loc_row.shape[1]*loc_row.shape[3])
        #print(summ_row)
        if type(summ_row) == int:
            summ_row = reshaped
        else:
            summ_row = summ_row + reshaped
print(summ_row.shape)


12
torch.Size([1, 23, 768])


In [93]:
print(len(output[1]))
trans_row = 0
output = output1
for row in output[1]:
    for loc_row in row:
        # shape of loc row is 1, a, b, c
        # reshape it to 1, b, a*c
        reshaped = loc_row.reshape(1, loc_row.shape[2], loc_row.shape[1]*loc_row.shape[3])
        #print(summ_row)
        if type(trans_row) == int:
            trans_row = reshaped
        else:
            trans_row = trans_row + reshaped
            trans_row = trans_row + reshaped
print(trans_row.shape)


12
torch.Size([1, 23, 768])


In [95]:
print(trans_row)

tensor([[[ -5.6148,  -2.3915,  -6.4956,  ...,  -6.9506, -29.2854,  38.3489],
         [-26.6708, -14.1379, -10.3205,  ...,  -1.1012,  -7.7927,  -3.1892],
         [  1.2218,  -1.7441,   1.6339,  ...,  -6.6021,  32.3533,  16.4269],
         ...,
         [ 19.2474,  -2.5872, -12.3756,  ...,   5.8171, -22.8726,  14.5963],
         [ -1.7809,  -2.5101,   0.8455,  ...,  24.0088,  23.7718,  16.6186],
         [  8.2050,  -3.4738,   7.5658,  ...,   7.9516,   7.3163,  11.7491]]],
       grad_fn=<AddBackward0>)


In [96]:
print(summ_row)

tensor([[[ -3.4352,   0.1803,  -2.4653,  ...,   0.0648, -14.0649,  14.0209],
         [-14.6782,  -1.5560,   1.7616,  ...,  -1.0546,  -3.2234,  -1.3001],
         [ -1.4422,  -2.0077,  -3.2644,  ...,  -4.5564,  15.1565,   9.5084],
         ...,
         [  5.4493,   1.9400,   1.8145,  ...,   0.2152,  -3.2449,   5.9530],
         [  1.8593,   1.7305,   2.3461,  ...,   2.4963,   4.0547,   9.8070],
         [  7.6833,   1.3099,  -2.8912,  ...,  10.4999,  17.7710,   2.8434]]],
       grad_fn=<AddBackward0>)
